# Flood mapping in Wales during 2020 winter

## Background

### Floodings in Wales during 2020

In 2020, Wales experienced some of the largest flooding on record. 
The sequencing, duration and severity of the rainfall in the catchment areas of Wales during February was of such intensity that many river levels reacted extremely quickly, reaching record levels and flows.
Records show 288mm of rain fell on average across Wales in February, but with some areas receiving up to four times the long-term monthly average.


For River Wye, Flood Warnings were issued on:
- 15 Feb (evening) - 17 Feb 2020
- 24 Feb - 25 Feb 2020
- 29 Feb - 01 Mar 2020
- 10 Mar 2020

For River Usk, Flood Alerts were issued on:
- 09 Feb - 10 Feb 2020
- 15 Feb (evening) - 18 Feb 2020
- 24 Feb - 25 Feb 2020
- 28 Feb  (evening) - 01 Mar 2020

For River Severn, Flood Warnings were issued on:
- 09 Feb - 10 Feb 2020
- 16 Feb - 17 Mar 2020
- 23 Feb - 25 Mar 2020
- 29 Feb 2020
- 10 Mar 2020

For River Dee, Flood Alerts were issued on:
- 09 Feb - 11 Feb 2020
- 13 Feb - 04 Mar 2020


![Image of Yaktocat](https://i.guim.co.uk/img/media/4c458a2a9007cb1d60a20a0cae70ab3439ed99d4/0_27_4314_2588/master/4314.jpg?width=620&quality=45&auto=format&fit=max&dpr=2&s=2371d95503d9d0f28a6c8206937f70ca)
An aerial view of the flooded Welsh town of Crickhowell, where the River Usk burst its banks as a result of Storm Dennis. Photograph: Christopher Furlong/Getty Images

## Description
This notebook demonstrates how floods can be rapidly mapped/monitored, using Sentinel-1 Synthetic Aperture Radar (SAR) satellite sensor through the framework of the newly developped Wales Data Cube (`WDC`). 

SAR is advantageous as it operates at wavelengths not impeded by cloud cover or a lack of illumination and data can be acquired over a site during day or night time and under all weather conditions. Sentinel-1 SAR operates at C-band (approximately 6 cm wavelength) and offers reliable repeated wide area monitoring.

The Living Wales project has faciliated the generation of national annual land cover maps (2017-2021), but is also actively involved in the development of a new digital infrastructure for Wales that provides Sentinel-1 data in an Analysis Ready Data (ARD) format.   These data allow observations of open water but also moisture in the environment, which can be used to map flood extent and dynamics.  The data are processed to a minimum set of requirements and organized into a form that allows immediate analysis with minimum additional user effort.  Historical and Near-Real Time products relating to flooding can be generated and also summarised over varying time periods. 

This notebook uses historical and NRT Sentinel-1 data, as well as a customed python library, to map flooding using algorithms developed through the Living Wales project. 

Topics include: 

1. Querying data for an area and period of interest (i.e., 01 Feb - 15 Mar 2020)
2. Loading data from datacube
3. Mapping Sentinel-1 signal for each date of the period of interest
4. Mapping flooded areas on daily basis
5. Mapping the progression of flood waters during the period of interest
6. Calculating water frequency


### Jupyter Notebooks
#### Running (executing) a cell
Jupyter Notebooks allow code to be separated into sections that can be executed independent of one another.
These sections are called "cells".

The python code is written into individual cells that can be executed by placing the cursor in the cell and typing `Shift-Enter` on the keyboard or selecting the &#9658; "Run" button in the ribbon at the top of the notebook.
These options will run a single cell at a time.

To automatically run all cells in a notebook, navigate to the "Cell" tab of the menu bar at the top of JupyterLab and select "Run All" (or the option that best suits your needs).
When a cell is run, the cell's content is executed.
Any output produced from running the cell will appear directly below it.

Run the cell below as a test:

In [ ]:
print("I ran a cell!")

#### Cell status
The `[ ]:` symbol to the left of each Code cell describes the state of the cell:

* `[ ]:` means that the cell has not been run yet.
* `[*]:` means that the cell is currently running.
* `[1]:` means that the cell has finished running and was the first cell run. The number indicates the order that the cells were run in.

***

## Getting started

To run this analysis, run all the cells in the notebook starting with the 'Load packages and connect to the datacube' cell.

### Load packages

Load key Python packages and supporting functions for the analysis, then connect to the datacube. 

In [ ]:
import sys
sys.path.append("..")

import datacube
import matplotlib.pyplot as plt
from time import time as time
import warnings
warnings.filterwarnings("ignore")
import wales_utils.data_cube_utilities.display_tools as display_tools


# Import Living Wales 'flooding' Python library
import wales_utils.themes_utilities.flooding as flooding

### Connect to the datacube
Connect to the datacube so we can access `WDC` Analysis Ready Data. 

In [ ]:
dc = datacube.Datacube(app="Flooding")

### Query an area and period
#### Area :  Please choose one of the four rivers, enter the name in the cell below and "Run" it
- 'wye'
- 'severn'
- 'dee'
- 'usk'


In [ ]:
#####################
#  ENTER RIVER NAME #
#####################

river = 'wye'

# select the above site
site = flooding.select_flooded_site(river)

#### Display the extent of the selected river site on ESRI World Imagery: `map_extent()`
World Imagery provides very high resolution (one meter or better) satellite and aerial imagery in many parts of the world. 
However, it is static imagery and, as a consequence, for each place there is only one date is available. 
For the study sites available this notebook, it is :
- River Wye: Maxar (Vivid) imagery captured 1 year 2 months ago, on 23 Apr 2021, with a 0.50 meters resolution
- Severn: Maxar (Vivid) imagery captured 2 years 3 months ago, on 7 Apr 2020, with a 0.50 meters resolution
- River Dee: Maxar (Vivid) imagery captured 3 years 9 months ago, on 27 Sept 2018, with a 0.50 meters resolution
- River Usk: Maxar (Vivid) imagery captured 2 years ago, on 25 Jun 2020, with a 0.50 meters resolution


This image is only used for visualising the extent of the selected study site and is not used in the following analyses.



In [ ]:
display_tools.map_extent(site)

#### Period
For this example, period of interest has been fixed to 01 Feb to 15 Mar 2020, which is the period of issued Flood Warnings/Alerts.

In [ ]:
start_date = '2020-02-01'
end_date = '2020-03-15'

print("Analysed period: " + start_date + " to " + end_date)

### Loading data from `WDC` 

#### Load data

In [ ]:
# Preparing query for datacube
min_x = site[0]
min_y = site[1]
max_x = site[2]
max_y = site[3]

query = {'x': (min_x, max_x),
         'y': (min_y, max_y),
         'crs':'epsg:27700',
         'time': (start_date, end_date),
         'output_crs': 'epsg:27700',
         'resolution': (-10,10)}

start_time = time()

# Let's load and clean a demo dataset
dataset_in = dc.load(product="sen1_rtc_pyroSNAP",
                     **query,
                    dask_chunks={'x': 1000, 'y': 1000})

# define nodata
dataset_in = dataset_in.where(dataset_in != 0)
# drop dates with only nodata for the area of interest
dataset_in = dataset_in.dropna('time', how='all')
# rename x and y with explicit names
dataset_in = dataset_in.rename({'x': 'longitude', 'y': 'latitude'})

# group images by YYYY-MM-DD
S1_dataset = dataset_in.groupby(dataset_in.time.dt.strftime("%Y-%m-%d")).mean("time")
S1_dataset = S1_dataset.rename({'strftime': 'time'})

print("Datacube ready")
print("Took only " + str(round(time()-start_time,2)) + " seconds to load a whole year of data from datacube \
for the specified extent (i.e., "+ str(len(dataset_in.time)*len(dataset_in.keys())) +" images).")

#### Have a look at the dataset structure

In [ ]:
S1_dataset

#### Map Sentinel-1 VH backscatter (decibel) for each date of the period of interest

In [ ]:
print("Plotting ...")
print("(Please wait until images appear. This may take a few seconds)")

S1_dataset.VH.plot(col='time', col_wrap=5);

### Mapping flooded areas
In this section, we will create a map of flood water extent for each date of the period of interest using the custom `WDC` `flood_mapping()` tool, and visualise them.

In [ ]:
# Map flooded areas for each available Sentinel-1 scene
floods = flooding.flood_mapping(S1_dataset)

In [ ]:
# Visualise flooded areas for each date
print("Plotting ...")
print("(Please wait until images appear. This may take a few seconds)")

floods.plot(col='time', col_wrap=5, add_colorbar=False, cmap="Blues");

### Mapping progression of flood waters

In this section, we will be mapping the progression of the flood waters through the landscape in Near-Real Time during the whole period using the custom `WDC` `flood_progression()` function. This allows sensing of (1) newly flooded areas, (2) areas where flood waters have receded and (3) areas still flooded, for each consecutive date.


- RED: newly flooded areas since previous satellite imagery
- ORANGE: areas remaining flooded
- BLUE: areas where flood waters have receded since previous satellite imagery

In [ ]:
# Calculate water progression between each date of the studied period 
water_changes = flooding.flood_progression(floods)

In [ ]:
# Visualise water progression between each date of the studied period 
print("Plotting ...")
print("(Please wait until images appear. This may take a few seconds)")

water_changes.plot(col='time', col_wrap=5, cmap="jet", add_colorbar=False);

### Calculate water frequency during February 2020
In this section, we will calculate how frequently each pixel was observed as wet during February 2020 using the custom `WDC` `flood_frequency()` function, and plot the frequency map.

To visualise results we are using the magma color scheme, but you can select a different one from the [Matplotlib library](https://matplotlib.org/stable/tutorials/colors/colormaps.html).

In [ ]:
# Calculate water frequency during February 2020
frequency = flooding.flood_frequency(floods, '2020-02-01', '2020-02-29')

In [ ]:
# Visualise water frequency during February 2020
print("Plotting ...")
print("(Please wait until images appear. This may take a few seconds)")

frequency.plot(figsize=(12,10), cmap="magma_r", levels=40);
plt.title('Water frequency during February 2020');

#### Overlay the result on an ESRI World Imagery

As part of the Living Wales project, we have added a tool (i.e, `display_da()`) in the `WDC` that allows overlaying an `xarray.DataArray` on a dynamic map.

Here we use the reverse magma colour scheme. You can select a different name of colour scheme from the [Matplotlib library](https://matplotlib.org/stable/tutorials/colors/colormaps.html).

In [ ]:
display_tools.display_da(frequency, colormap='magma_r')

Let's re-run the whole analysis for another river (e.g., 'servern'). Change the name of the river in cell `4`, and then navigate to `Kernel` and select `'Restart Kernel and Run All Cells...'`